In [1]:
import os

In [2]:
%pwd

'd:\\ML PROJECTS\\Chest-cancer-classification-MLflow-dvc\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML PROJECTS\\Chest-cancer-classification-MLflow-dvc'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [19]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Dowwnloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-22 11:40:17,454: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2026-01-22 11:40:17,457: INFO: common: yaml file: params.yaml loaded succesfully]
[2026-01-22 11:40:17,459: INFO: common: created directory at: artifacts]
[2026-01-22 11:40:17,461: INFO: common: created directory at: artifacts/data_ingestion]
[2026-01-22 11:40:17,463: INFO: 481082023: Dowwnloading data from https://drive.google.com/file/d/1D_ekTLlJm2SWYQ9J1Q5pGdhxVY5E_99X/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1D_ekTLlJm2SWYQ9J1Q5pGdhxVY5E_99X
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1D_ekTLlJm2SWYQ9J1Q5pGdhxVY5E_99X&confirm=t&uuid=62d31a3f-6ce5-4259-9acb-45c807537627
To: d:\ML PROJECTS\Chest-cancer-classification-MLflow-dvc\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [00:05<00:00, 22.7MB/s] 

[2026-01-22 11:40:27,218: INFO: 481082023: Download data from https://drive.google.com/file/d/1D_ekTLlJm2SWYQ9J1Q5pGdhxVY5E_99X/view?usp=sharing into file artifacts/data_ingestion/data.zip]
